In [1]:
import os
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.decomposition import PCA

# === CONFIG ===
obj_file = r'C:\Users\akhil\OneDrive\Documents\point2building\RoofVE-main\wireframe\10068.obj'  # replace with your test .obj
output_csv = 'test_data_features.csv'

# === UTILITY FUNCTIONS ===

def cos_angle(v1, v2):
    if np.linalg.norm(v1) == 0 or np.linalg.norm(v2) == 0:
        return 0
    return np.clip(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)), -1, 1)

def distance(v1, v2):
    return np.linalg.norm(np.array(v1) - np.array(v2))

def compute_curvature(vertex, neighbors, vertices):
    if len(neighbors) < 2:
        return 0
    angles = []
    for i in range(len(neighbors)):
        for j in range(i + 1, len(neighbors)):
            v1 = np.array(vertices[neighbors[i]]) - np.array(vertex)
            v2 = np.array(vertices[neighbors[j]]) - np.array(vertex)
            angle = np.arccos(cos_angle(v1, v2))
            angles.append(angle)
    return np.mean(angles) if angles else 0

def compute_normal(vertex, neighbors, vertices):
    if len(neighbors) < 2:
        return np.array([0, 0, 0])
    v1 = np.array(vertices[neighbors[0]]) - np.array(vertex)
    v2 = np.array(vertices[1]) - np.array(vertex)
    normal = np.cross(v1, v2)
    norm = np.linalg.norm(normal)
    return normal / norm if norm > 0 else np.array([0, 0, 0])

def compute_planarity(vertex, neighbors, vertices):
    if len(neighbors) < 3:
        return 0
    points = np.array([vertices[n] for n in neighbors])
    pca = PCA(n_components=3)
    pca.fit(points)
    eigenvalues = sorted(pca.explained_variance_, reverse=True)
    return (eigenvalues[1] - eigenvalues[2]) / eigenvalues[0] if eigenvalues[0] > 0 else 0

# === PARSE OBJ ===

vertices = []
edges = []
vertex_neighbors = {}

filename = os.path.splitext(os.path.basename(obj_file))[0]

with open(obj_file, 'r') as file:
    for line in file:
        if line.startswith('v '):
            vertex = [float(x) for x in line.strip().split()[1:4]]
            vertices.append(vertex)
            vertex_neighbors[len(vertices) - 1] = []
        elif line.startswith('l '):
            v1, v2 = [int(i) - 1 for i in line.strip().split()[1:]]
            edges.append((v1, v2))
            vertex_neighbors[v1].append(v2)
            vertex_neighbors[v2].append(v1)

edges_set = set(tuple(sorted(e)) for e in edges)

# === FEATURE EXTRACTION ===

features = []

for v1, v2 in combinations(range(len(vertices)), 2):
    coord1 = vertices[v1]
    coord2 = vertices[v2]

    dist = distance(coord1, coord2)
    height_diff = abs(coord1[2] - coord2[2])
    
    normal1 = compute_normal(coord1, vertex_neighbors[v1], vertices)
    normal2 = compute_normal(coord2, vertex_neighbors[v2], vertices)
    normal_similarity = cos_angle(normal1, normal2)

    curvature1 = compute_curvature(coord1, vertex_neighbors[v1], vertices)
    curvature2 = compute_curvature(coord2, vertex_neighbors[v2], vertices)

    planarity1 = compute_planarity(coord1, vertex_neighbors[v1], vertices)
    planarity2 = compute_planarity(coord2, vertex_neighbors[v2], vertices)

    label = 1 if (min(v1, v2), max(v1, v2)) in edges_set else 0

    features.append([
        filename, v1 + 1, v2 + 1,
        dist, height_diff, normal_similarity,
        planarity1, curvature1, curvature2, planarity2, label
    ])

# === SAVE CSV ===

df = pd.DataFrame(features, columns=[
    'filename', 'v1', 'v2', 'distance', 'height_diff', 'normal_similarity',
    'planarity_v1', 'curvature_v1', 'curvature_v2', 'planarity_v2', 'label'
])
df.to_csv(output_csv, index=False)
print(f"Test features saved to {output_csv}")


Test features saved to test_data_features.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
df = pd.read_csv(r'C:\Users\akhil\OneDrive\Documents\point2building\RoofVE-main\balanced_dataset.csv')  # Change to your actual filename if different

# Separate features and label
X = df.drop(columns=['label'])
y = df['label']

# Stratified split into 80% train and 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Combine features and labels again
train_df = X_train.copy()
train_df['label'] = y_train

test_df = X_test.copy()
test_df['label'] = y_test

# Save to CSV
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)

# Optional: Print confirmation
print("✅ Files saved as 'train.csv' and 'test.csv'")
print("\n🔍 Test Set Label Distribution:\n", test_df['label'].value_counts(normalize=True))


✅ Files saved as 'train.csv' and 'test.csv'

🔍 Test Set Label Distribution:
 label
0    0.545444
1    0.454556
Name: proportion, dtype: float64


In [ ]:
# Load data from your .txt file
input_path = 'corner_vertices.txt'   # ⬅️ Change to your actual file path
output_path = 'converted_vertices.obj'

with open(input_path, 'r') as f:
    lines = f.readlines()

# Convert and write to .obj format
with open(output_path, 'w') as f_out:
    for line in lines:
        if line.strip():  # skip empty lines
            x, y, z = map(float, line.strip().split(','))
            f_out.write(f"v {x} {y} {z}\n")

print(f"✅ Converted coordinates saved to: {output_path}")


In [9]:
import open3d as o3d

# Path to your .ply file
ply_path = r"C:\Users\akhil\OneDrive\Documents\Building_recon\building_clusters_2\building_1.ply"  # ⬅️ Replace with your actual file path

# Load the point cloud
pcd = o3d.io.read_point_cloud(ply_path)

# Set uniform color (R, G, B) — here blue
pcd.paint_uniform_color([0.1, 0.1, 0.9])  # RGB values between 0 and 1

# Print info
print(pcd)
print("Displaying point cloud with color ...")

# Visualize
o3d.visualization.draw_geometries([pcd])


PointCloud with 4337 points.
Displaying point cloud with color ...


In [ ]:
import open3d as o3d
import numpy as np

# Path to your .txt file (with comma-separated XYZ coordinates)
txt_path = r"C:\Users\akhil\OneDrive\Documents\point2building\RoofVE-main\pred_res\10007_finCor_Geo.txt"

# Load the points (comma-separated)
points = np.loadtxt(txt_path, delimiter=",", dtype=np.float64)

# Create point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# Compute convex hull
hull, _ = pcd.compute_convex_hull()
hull.paint_uniform_color([0.2, 0.6, 0.9])  # light blue color

# Optional: Wireframe from convex hull
edges = o3d.geometry.LineSet.create_from_triangle_mesh(hull)

# Visualize
o3d.visualization.draw_geometries([pcd, edges], window_name="Building Wireframe")




FileNotFoundError: C:\Users\akhil\OneDrive\Documents\point2building\RoofVE-main\pred_res\building_10007_finCor_Geo.txt not found.

In [33]:
import open3d as o3d
import numpy as np

# === Load point cloud from .txt ===
txt_path = r"C:\Users\akhil\OneDrive\Documents\point2building\RoofVE-main\pred_res\cluster_building_cluster_117_sub_11_finCor_Geo.txt"
points = np.loadtxt(txt_path, delimiter=",", dtype=np.float64)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

# === Compute convex hull ===
hull, _ = pcd.compute_convex_hull()
hull.paint_uniform_color([0.2, 0.6, 0.9])  # Light blue

# === Create edges from convex hull mesh ===
lineset = o3d.geometry.LineSet.create_from_triangle_mesh(hull)

# === Remove low-height & base edges ===
vertices = np.asarray(hull.vertices)
lines = np.asarray(lineset.lines)

height_threshold = 0.3 # Minimum height diff between two edge points
ground_threshold = 1    # Anything at or below this z-value is base

filtered_lines = []
for line in lines:
    pt1 = vertices[line[0]]
    pt2 = vertices[line[1]]
    z1, z2 = pt1[2], pt2[2]
    height_diff = abs(z1 - z2)

    # Keep if it's not a base line and has sufficient height difference
    if height_diff >= height_threshold and not (z1 <= ground_threshold and z2 <= ground_threshold):
        filtered_lines.append(line)

# Create new LineSet with filtered edges
filtered_lineset = o3d.geometry.LineSet()
filtered_lineset.points = o3d.utility.Vector3dVector(vertices)
filtered_lineset.lines = o3d.utility.Vector2iVector(filtered_lines)
filtered_lineset.paint_uniform_color([1, 0, 0])  # Red edges
# === Save filtered wireframe ===
save_path = r"C:\Users\akhil\OneDrive\Documents\point2building\RoofVE-main\pred_res\cluster_building_cluster_117_sub_11_4finCor_wireframe.ply"
o3d.io.write_line_set(save_path, filtered_lineset)
print(f"Wireframe saved to: {save_path}")

# === Visualize ===
o3d.visualization.draw_geometries(
    [pcd, filtered_lineset],
    window_name="wireframe"
)




Wireframe saved to: C:\Users\akhil\OneDrive\Documents\point2building\RoofVE-main\pred_res\cluster_building_cluster_117_sub_11_4finCor_wireframe.ply


In [ ]:
import pandas as pd
import numpy as np

# Load .txt file (comma-separated)
points_df = pd.read_csv(r"C:\Users\akhil\OneDrive\Documents\point2building\RoofVE-main\pred_res\building_1_finCor_Geo.txt", header=None, names=['x', 'y', 'z'])

# Convert to numpy array
points = points_df[['x', 'y', 'z']].values


In [9]:
from sklearn.neighbors import NearestNeighbors

# Use KNN to find nearest neighbors (e.g., k=3)
k = 3
nn = NearestNeighbors(n_neighbors=k+1, algorithm='auto').fit(points)
distances, indices = nn.kneighbors(points)

edges = []

for i, neighbors in enumerate(indices):
    for j in neighbors[1:]:  # skip self
        if i < j:  # avoid duplicate edges
            edges.append((i, j))

# Now each edge is (index1, index2)


In [ ]:
def compute_edge_features(p1, p2):
    vec = p2 - p1
    distance = np.linalg.norm(vec)
    height_diff = abs(p2[2] - p1[2])
    
    # Placeholder for other features
    return {
        'cos_angle_v1': 0,
        'cos_angle_v2': 0,
        'curvature_v1': 0,
        'curvature_v2': 0,
        'angle_pos_v1_v2': 0,
        'distance': distance,
        'normal_similarity': 0,
        'planarity_v1': 0,
        'planarity_v2': 0,
        'height_diff': height_diff,
        'convexity': 0
    }

features = []

for i1, i2 in edges:
    p1 = points[i1]
    p2 = points[i2]
    f = compute_edge_features(p1, p2)
    features.append(f)

features_df = pd.DataFrame(features)


In [ ]:
from sklearn.impute import SimpleImputer
selected_features = [
    'distance',
    'planarity_v1',
    'height_diff',
    'normal_similarity',
    'curvature_v2',
    'planarity_v2',
    'curvature_v1'
]


imputer = SimpleImputer(strategy='mean')
features_df[selected_features] = imputer.fit_transform(features_df[selected_features])


In [24]:
import joblib

# Load trained model
model = joblib.load(r"C:\Users\akhil\OneDrive\Documents\point2building\lightgbm_edge_detector.pkl")

# Predict
preds = model.predict(features_df[selected_features])
labels = (preds > 1).astype(int)

features_df['predicted_edge'] = labels


In [25]:
import open3d as o3d
import numpy as np

# Load the point coordinates
points = np.loadtxt(r'C:\Users\akhil\OneDrive\Documents\point2building\RoofVE-main\pred_res\building_1030_finCor_Geo.txt', delimiter=',')

# Load predicted labels (0 or 1)
# Assuming you already have this as a column in `features_df`
labels = features_df['predicted_edge'].values
# Create open3d PointCloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.paint_uniform_color([0.5, 0.5, 0.5])  # gray points

# Create LineSet to represent edges
lines = []
colors = []

for (idx, (i, j)) in enumerate(edges):
    lines.append([i, j])
    if labels[idx] == 1:
        colors.append([1, 0, 0])  # red edge (predicted)
    else:
        colors.append([0.7, 0.7, 0.7])  # gray edge

# Convert to Open3D LineSet
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(points)
line_set.lines = o3d.utility.Vector2iVector(lines)
line_set.colors = o3d.utility.Vector3dVector(colors)

# Visualize
o3d.visualization.draw_geometries([pcd, line_set])


In [27]:
import pandas as pd
import numpy as np
import joblib
import open3d as o3d
from itertools import combinations
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# === 1. Load corner points from .txt ===
points = np.loadtxt(r"C:\Users\akhil\OneDrive\Documents\point2building\RoofVE-main\pred_res\building_1030_finCor_Geo.txt", delimiter=",")  # Replace with your actual path
print("Loaded points shape:", points.shape)

# === 2. Create all unique edge pairs ===
edges = list(combinations(range(len(points)), 2))  # All possible pairs (i, j)

# === 3. Feature extraction for each edge ===
def extract_features(p1, p2):
    vec = p2 - p1
    distance = np.linalg.norm(vec)
    height_diff = abs(p2[2] - p1[2])
    planarity_v1 = vec[2] / (distance + 1e-6)  # Simplified proxy
    normal_similarity = 1.0  # Placeholder
    curvature_v1 = abs(planarity_v1)
    planarity_v2 = planarity_v1  # Same for now
    curvature_v2 = curvature_v1
    return [distance, planarity_v1, height_diff, normal_similarity,
            curvature_v2, planarity_v2, curvature_v1]

# Extract features for all edges
features = [extract_features(points[i], points[j]) for i, j in edges]
features_df = pd.DataFrame(features, columns=[
    'distance', 'planarity_v1', 'height_diff', 'normal_similarity',
    'curvature_v2', 'planarity_v2', 'curvature_v1'
])

# === 4. Impute and Scale ===
imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()
features_df = pd.DataFrame(imputer.fit_transform(features_df), columns=features_df.columns)
features_df = pd.DataFrame(scaler.fit_transform(features_df), columns=features_df.columns)

# === 5. Load model and predict ===
model = joblib.load(r"C:\Users\akhil\OneDrive\Documents\point2building\lightgbm_edge_detector.pkl")
preds = model.predict(features_df[['distance', 'planarity_v1', 'height_diff',
                                   'normal_similarity', 'curvature_v2', 'planarity_v2', 'curvature_v1']])
labels = (preds > 0.2).astype(int)

# === 6. Visualize predicted edges ===
lines = [edges[i] for i in range(len(edges)) if labels[i] == 1]

line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(points)
line_set.lines = o3d.utility.Vector2iVector(lines)

# Color edges red
colors = [[1, 0, 0] for _ in lines]
line_set.colors = o3d.utility.Vector3dVector(colors)

# Also show the corner points
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.paint_uniform_color([0.5, 0.5, 0.5])  # Gray for corners

# Visualize
o3d.visualization.draw_geometries([pcd, line_set])


Loaded points shape: (22, 3)
